# Importing Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import urllib.request
import ipyplot
from datetime import date, datetime, timedelta

# Defining Functions

### Defining API Strings

In [2]:
magic_eden_base_url = "http://api-mainnet.magiceden.dev/v2"
binance_base_url = "https://api.binance.com"
solscan_base_url = "https://public-api.solscan.io"

### Get Collection Stats

In [3]:
# Purpose: Gets the basic stats of a collection
# Accepts: collection symbol on ME ("the_whitelist_project")
# Returns: [listed count, floor price, all-time volume]

def get_collection_stats(collection_symbol):
    
    endpoint = "/collections/{}/stats".format(collection_symbol)
    
    querystring = {"":""}
    response = requests.request("GET", magic_eden_base_url+endpoint, params=querystring)
    response_json = response.json()
    
    listed_count = response_json['listedCount']
    floor_price = round(response_json['floorPrice'] / 1000000000, 2)
    total_volume = round(response_json['volumeAll'] / 1000000000, 2)
    
    return [listed_count, floor_price, total_volume]

### Get Upcoming Launchpad Mints

In [4]:
# Purpose: Gets upcoming ME launchpad collections
# Accepts: nothing
# Returns: list of dictionaries - all launchpad collections that haven't minted yet

def get_upcoming_launchpad_mints():
    
    endpoint = "/launchpad"
    
    querystring = {"offset":"0", "limit":"500"}
    response = requests.request("GET", magic_eden_base_url+endpoint, params=querystring)
    response_json = response.json()

    upcoming_mints = []

    for collection in response_json:
    
        if collection.get('launchDatetime') is not None:
        
            launch_date = datetime.strptime(collection.get('launchDatetime')[:10], '%Y-%m-%d')
            today = datetime.strptime(datetime.strftime(datetime.today(), '%Y-%m-%d'), '%Y-%m-%d')
            lead_time = timedelta(days=0)
        
            if launch_date - today > lead_time:
            
                upcoming_mints.append(collection)   
        
    return upcoming_mints

### Track Wallet Activities

In [5]:
# Purpose: Stalk whale wallets and stuff
# Accepts: wallet address(es)?
# Returns: list of dictionaries - latest 500 buys and sells from the specified wallet address

def get_wallet_activity(wallet_address):
    
    endpoint = "/wallets/{}/activities".format(wallet_address)
    
    querystring = {"offset":"0", "limit":"500"}
    response = requests.request("GET", magic_eden_base_url+endpoint, params=querystring)
    response_json = response.json()
    
    buys_and_sells = []
    
    for action in response_json:
        
        if (action.get('type') == 'buyNow' or action.get('type') == 'list'):
            
            buys_and_sells.append(action)
    
    return buys_and_sells

### Get Coin Prices

In [6]:
# Purpose: Get price data for a handful of cryptocurrencies
# Accepts: nothing
# Returns: list of dictionaries - BTC, ETH, and SOL price data

def get_crypto_prices():
    
    # couldn't get the multiple coin API to work - doing each of the 3 manually
    
    coin_prices = []
    
    # btc stats
    btc_endpoint = "/api/v3/ticker/24hr?symbol=BTCUSDC"
    querystring = {}
    response = requests.request("GET", binance_base_url+btc_endpoint, params=querystring)
    btc_response_json = response.json()
    
    coin_prices.append({"name":"Bitcoin","price":btc_response_json.get("lastPrice"),
                        "% Change 24h":btc_response_json.get("priceChangePercent")})
    
    # eth stats
    eth_endpoint = "/api/v3/ticker/24hr?symbol=ETHUSDC"
    querystring = {}
    response = requests.request("GET", binance_base_url+eth_endpoint, params=querystring)
    eth_response_json = response.json()
    
    coin_prices.append({"name":"Ethereum","price":eth_response_json.get("lastPrice"),
                        "% Change 24h":eth_response_json.get("priceChangePercent")})
    
    # sol stats
    sol_endpoint = "/api/v3/ticker/24hr?symbol=SOLUSDC"
    querystring = {}
    response = requests.request("GET", binance_base_url+sol_endpoint, params=querystring)
    sol_response_json = response.json()
    
    coin_prices.append({"name":"Solana","price":sol_response_json.get("lastPrice"),
                        "% Change 24h":sol_response_json.get("priceChangePercent")})

    return coin_prices  

### Token Information

In [7]:
# Purpose: Gets information about an NFT
# Accepts: NFT token address ("67a7RcsvttpTjUN44iMcHwC4HMJSXcuPYAt4GaBaxXu6")
# Returns: NFT details

def get_collection_stats(token):
    
    endpoint = "/tokens/{}".format(token)
    
    querystring = {"":""}
    response = requests.request("GET", magic_eden_base_url+endpoint, params=querystring)
    response_json = response.json()
    
    return response_json

In [9]:
print(get_collection_stats("67a7RcsvttpTjUN44iMcHwC4HMJSXcuPYAt4GaBaxXu6"))

{'mintAddress': '67a7RcsvttpTjUN44iMcHwC4HMJSXcuPYAt4GaBaxXu6', 'owner': 'CMDScdvqB3ZJJHVv7HbP3yJcmQCs25mEXfXv4BBfXLcH', 'supply': 1, 'collection': 'the_whitelist_project', 'name': 'Whitelist Ticket #967', 'updateAuthority': '9yPET7vSL7t27zFAevrni2xe3HJ6MuHQ6mNoaKLXsj6j', 'primarySaleHappened': True, 'sellerFeeBasisPoints': 500, 'image': 'https://www.arweave.net/OiRGvlvGLRwMij8OuZwbtFBelOcjroQcBllu0MPEews?ext=png', 'attributes': [{'trait_type': 'Background', 'value': 'Off-White[list]'}, {'trait_type': 'Text', 'value': 'Wavy'}, {'trait_type': 'Ticket Type', 'value': 'Priority'}, {'trait_type': 'Stripe', 'value': 'None'}, {'trait_type': 'Stamp', 'value': 'Postage'}, {'trait_type': 'Extras', 'value': 'None'}], 'properties': {'category': 'image', 'creators': [{'address': 'GWgmxJZjtNMyLLuMj7y8WESFT8BxXU8cmVnWSwYujxxe', 'share': 100}], 'files': [{'uri': 'https://www.arweave.net/OiRGvlvGLRwMij8OuZwbtFBelOcjroQcBllu0MPEews?ext=png', 'type': 'image/png'}]}}


## Next Steps

- Automatically kick off on certain intervals
- Integrate with a Discord bot